In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
from collections import namedtuple, defaultdict
import inspect
import configparser
from prettytable import PrettyTable

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [149]:
config = configparser.ConfigParser()
config.read('../config.ini')
config_dict = {section: dict(config[section]) for section in config.sections()}


In [168]:
model_parameters = {
    "max_iter": int(config["CLASSIFICATION"]["maxiterations"]),
    "cv": int(config["CLASSIFICATION"]["cvfolds"]),
    "tolerance": float(config["CLASSIFICATION"]["tolerance"]),
    "random_state": int(config["CLASSIFICATION"]["randomstate"]),
    "verbose": bool(config["CLASSIFICATION"]["verbose"]),
}

model_parameters = defaultdict(lambda: None, model_parameters)

# Simple Classification Models

## Binary Classification

1. `LogisticRegression`: Logistic Regression (aka logit, MaxEnt) classifier.

2. `SGDClassifier`: Linear classifiers (SVM, logistic regression, etc.) with SGD training.

3. `Perceptron`: The perceptron is a simple classification algorithm suitable for large scale learning.

4. `PassiveAggressiveClassifier`: Passive Aggressive Classifier.

5. `RidgeClassifier`: Classifier using Ridge regression.

6. `RidgeClassifierCV`: Ridge classifier with built-in cross-validation.

7. `LinearSVC`: Linear Support Vector Classification.

8. `SVC`: C-Support Vector Classification.

9. `NuSVC`: Nu-Support Vector Classification.

10. `DecisionTreeClassifier`: A decision tree classifier.

11. `RandomForestClassifier`: A random forest classifier.

12. `ExtraTreesClassifier`: An extra-trees classifier.

13. `GradientBoostingClassifier`: Gradient Boosting for classification.

14. `HistGradientBoostingClassifier`: Histogram-based Gradient Boosting Classification Tree.

15. `AdaBoostClassifier`: An AdaBoost classifier.

16. `BaggingClassifier`: A Bagging classifier.

17. `VotingClassifier`: Soft Voting/Majority Rule classifier for unfitted estimators.

18. `StackingClassifier`: Stacking classifier for unfitted estimators.

19. `KNeighborsClassifier`: Classifier implementing the k-nearest neighbors vote.

20. `RadiusNeighborsClassifier`: Classifier implementing a vote among neighbors within a given radius.

21. `MLPClassifier`: Multi-layer Perceptron classifier.

22. `GaussianNB`: Gaussian Naive Bayes (GaussianNB).

23. `BernoulliNB`: Naive Bayes classifier for multivariate Bernoulli models.

24. `ComplementNB`: The Complement Naive Bayes classifier.

25. `MultinomialNB`: Naive Bayes classifier for multinomial models.

Please refer to the [Scikit-learn documentation](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) for more details on each of these methods.


### Data Prep

In [39]:
from sklearn.datasets import load_breast_cancer

breast_cancer_df = load_breast_cancer(as_frame=True).frame

y = breast_cancer_df.pop('target').values
X_df = breast_cancer_df

scaler = StandardScaler()

for column in X_df.columns:
    X_df[column] = scaler.fit_transform(X_df[column].values.reshape(-1, 1))

In [43]:
train_X, test_X, train_y, test_y = train_test_split(X_df.values, y, test_size=0.3, random_state=42)
test_X, val_X, test_y, val_y = train_test_split(test_X, test_y, test_size=0.5, random_state=42)

### Running Models

In [88]:
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier, Perceptron, PassiveAggressiveClassifier, RidgeClassifierCV
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier, VotingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.neural_network import MLPClassifier

In [169]:
def add_appropriate_kwargs(function):
    signature = inspect.signature(function)
    parameters = signature.parameters
    keywords = {k: v.default for k, v in parameters.items() if v.default is not inspect.Parameter.empty}

    for key, _ in keywords.items():
        if key in model_parameters:
            keywords[key] = model_parameters[key]

    return partial(function, **keywords)        

In [203]:
model_and_metrics = namedtuple('model_and_metrics', ['model', 'accuracy', 'log_loss', 'confusion_matrix'])

def train_model(model, train_X, train_y, test_X, test_y):
    print("-------------------")
    print("training model: ", model.__class__.__name__)
    model.fit(train_X, train_y)
    pred_y = model.predict(test_X)
    return model_and_metrics(model, accuracy_score(test_y, pred_y), log_loss(test_y, pred_y), cm)

models = [
    LogisticRegressionCV,
    SGDClassifier,
    Perceptron,
    PassiveAggressiveClassifier,
    RidgeClassifierCV,
    LinearSVC,
    SVC,
    NuSVC,
    DecisionTreeClassifier,
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    KNeighborsClassifier,
    MLPClassifier,
    GaussianNB,
    BernoulliNB,
]

curried_models = [add_appropriate_kwargs(model) for model in models]




In [205]:
all_models = [train_model(model(), train_X, train_y, test_X, test_y) for model in curried_models]

-------------------
training model:  LogisticRegressionCV
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.20421D+02    |proj g|=  1.17153D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     10     13      1     0     0   3.691D-04   2.015D+02
  F =   201.45698504626739     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 varia

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



At iterate  100    f=  8.52725D-01    |proj g|=  3.86270D-04

At iterate  150    f=  8.52637D-01    |proj g|=  1.34924D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31    154    165      1     0     0   9.137D-05   8.526D-01
  F =  0.85263478619472211     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.20421D+02    |proj g|=  1.20801D+02

           * * *

Tit   = total number of iterations
Tnf   = total nu

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     40     45      1     0     0   2.549D-04   2.212D+01
  F =   22.123863891693539     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.85621D+01    |proj g|=  5.53432D-01

At iterate   50    f=  1.51731D+01    |proj g|=  1.70012D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments 

...

-------------------
training model:  LinearSVC
[LibLinear].........................................*...................*
optimization finished, #iter = 608
Objective value = -21.193602
nSV = 47
-------------------
training model:  SVC
[LibSVM]*
optimization finished, #iter = 147
obj = -47.321991, rho = -0.315321
nSV = 93, nBSV = 51
Total nSV = 93
-------------------
training model:  NuSVC
[LibSVM]*
optimization finished, #iter = 136
C = 0.073922
obj = 4.034342, rho = -0.225667
nSV = 205, nBSV = 192
Total nSV = 205
-------------------
training model:  DecisionTreeClassifier
-------------------
training model:  RandomForestClassifier


/Users/richardstrange/opt/miniconda3/envs/ml-portfolio/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


-------------------
training model:  ExtraTreesClassifier


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


-------------------
training model:  GradientBoostingClassifier
      Iter       Train Loss   Remaining Time 
         1           1.1498            0.83s
         2           1.0119            0.80s
         3           0.8978            1.00s
         4           0.8023            1.02s
         5           0.7197            1.04s
         6           0.6475            1.06s
         7           0.5855            1.07s
         8           0.5326            1.04s
         9           0.4851            1.02s
        10           0.4442            1.00s
        20           0.1980            0.77s
        30           0.1026            0.64s
        40           0.0537            0.53s
        50           0.0316            0.43s
        60           0.0191            0.34s
        70           0.0120            0.25s
        80           0.0075            0.17s
        90           0.0051            0.08s
       100           0.0034            0.00s
-------------------
training model:

In [210]:
table = PrettyTable()
table.field_names = ["Model", "Accuracy", "Log Loss"]
for model in all_models:
    table.add_row([model.model.__class__.__name__, model.accuracy, model.log_loss])

print(table)

+--------------------------------+--------------------+--------------------+
|             Model              |      Accuracy      |      Log Loss      |
+--------------------------------+--------------------+--------------------+
|      LogisticRegressionCV      | 0.9882352941176471 | 0.4240429810484373 |
|         SGDClassifier          | 0.9764705882352941 | 0.8480859620968744 |
|           Perceptron           | 0.9294117647058824 | 2.5442578862906227 |
|  PassiveAggressiveClassifier   | 0.9764705882352941 | 0.8480859620968744 |
|       RidgeClassifierCV        | 0.9647058823529412 | 1.2721289431453116 |
|           LinearSVC            | 0.9764705882352941 | 0.8480859620968744 |
|              SVC               | 0.9529411764705882 | 1.6961719241937483 |
|             NuSVC              | 0.9529411764705882 | 1.6961719241937483 |
|     DecisionTreeClassifier     | 0.9411764705882353 | 2.1202149052421855 |
|     RandomForestClassifier     | 0.9647058823529412 | 1.2721289431453116 |